In [1]:
import wandb
import pandas as pd
import pandas_profiling

In [2]:
# Retrieve the latest data artifact
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()

wandb: Currently logged in as: cmiranda (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
df = pd.read_csv(local_path)

In [4]:
profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/29 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error resolved after 0:00:08.300122, resuming normal operation.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error resolved after 0:00:51.813063, resuming normal operation.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error resolved after 0:01:30.910078, resuming normal operation.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error resolved after 0:03:42.207745, resuming normal operation.


Identification features:
- `id` is unique (listing identifier)
- `name` values can include interesting information about the location and broader description of the listings. There are some repeated values, could those be the same properties listed multiple times ?
- `host_id` counts presents some outliers, that seems to be hosts that post >= 30 properties. These are an internal identifiers, so clients should not be influenced by it, but the feature could be correlated to the "seniority" of the host using the platform, which could impact client decisions.
- `host_name`, like `host_id` is a high cardinality variable which does not seem to correlate with other variables, besides `host_id` evidently.
- `calculated_host_listings_count` 

Spatial features:
- `neighbourhood_group` is very skewed, with two values being 85% (manhattan and brooklyn).
- `neighbourhood` is more specific than neighbourhood group and should be positively correlated with it.
- `longitude` and `latitude` are correlated with `neighbourhood[_group]` variables (kind of obvious), but it seems `latitude` does not show significative correlation with price, while `longitude` shows negative correlation (Pearson's, Spearman's and Kendall's coefficients).

Property features:
- Mainly two categories for `room_type`, and there seems to not be significant correlation with any other feature.
- `price` has some extreme values (95 percentile = 350 while 5 larger values > 7000) which could be damaging the bivariate analyses. As is, `price` seems not significantly correlated with any other variable.
- `minimum_nights` shows some subjectively weird values (e.g. 370, 1250) and so it is highly skewed. From the bivariate analysis, it would seem properties with higher minimum nights tend to get fewer reviews. Intuitively, higher minimum nights implies fewer guests, which implies fewer reviews.
- `availability_365`


In [6]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()

# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   name                            18994 non-null  object        
 2   host_id                         19001 non-null  int64         
 3   host_name                       18993 non-null  object        
 4   neighbourhood_group             19001 non-null  object        
 5   neighbourhood                   19001 non-null  object        
 6   latitude                        19001 non-null  float64       
 7   longitude                       19001 non-null  float64       
 8   room_type                       19001 non-null  object        
 9   price                           19001 non-null  int64         
 10  minimum_nights                  19001 non-null  int64         
 11  nu

In [54]:
run.finish()